# Install Requirements

In [ ]:
pip install -r requirements.txt

# Load Env Vars

In [ ]:
import dotenv

dotenv.load_dotenv(override=True)

# State Model

Frist of all, we need to define the `State` of the graph, that is all the variables that are shared between the nodes.

The `State` class defines all the attributes of the graph state, i.e. the elements all the nodes can access, use and modify, including the user question, the knowledge and the answer.  

The state attributes in this case are:
- `query`: the user query. It is used in the graph to read the user question.
- `messages`: the list of intermediate steps of the agent's actions and observations. It contains the choices made by the model on which tool (if any) to use, the tool result and eventually also the final answer.

In [ ]:
from pydantic import BaseModel
from typing import Annotated, List

from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


class State(BaseModel):
    """
    Custom graph state to support agent-oriented decision making.
    During each step, each node is able to add, modify, or extract the values from the state object.

    Attributes:
    - query (str): The user's question. Read-only.
    - messages (List[AnyMessage]): The list of intermediate steps of the agent's actions and observations.
            It will also contain the final answer.
        NOTE: Annotated with add_messages to allow for adding steps to the agent-tools conversation.
    """

    query: str
    messages: Annotated[List[AnyMessage], add_messages]

# Load Index

The `allow_dangerous_deserialization` parameter allows to load the index pikle file. Set it to `True` only to open trusted files, or files that you have created.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


vectorstore = FAISS.load_local(
    "pdf_index", OpenAIEmbeddings(), allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# Nodes Definition

## Tools Definition

In the tool definition, add a docstring that explains what the tool does and what the input is. The docstring will be included in the model calls so the model knows what all the tools are meant for and how to call them.

In [ ]:
from langchain_core.tools import tool


@tool("retrieve")
def retrieve(query: str):
    """
    Query 'pdf_index' that contains info on TOPIC.
    
    Args:
        - query (str): The search query.
    """
    
    docs = retriever.invoke(query)
    return "\n\n".join([doc.page_content for doc in docs])

## Chain Definition

In [ ]:
AGENT_PROMPT = """You are a helpful assistant.
Your task is to answer questions about TOPIC.

"""

Some notes on tool calling:

- **Tool Creation**: use the tool function to create a tool, that is an association between a function and its schema.

- **Tool Binding**: the tool is then connected to the model that supports tool calling (eg `gpt-4o`), so that the model is aware of the tool and the associated input schema. The method `.bind_tools()` is used to pass tool schemas to the model in subsequent invocations of the model.

- **Tool Calling**: the model can decide to call the tool.

- **Tool Execution**: the tool is executed with the arguments provided by the model.


**NOTE**: OpenAI models allow for parallel tool calling by default. this means that the model can call multiple tools at a time. If you prefer to avoid this behaviour and force the model to call one tool at a time, you have to include the option 

```python
parallel_tool_calls = False
```

when binding tools.

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", AGENT_PROMPT),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("human", "{question}"),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([retrieve], parallel_tool_calls=False)
agent_chain = prompt | llm_with_tools

## Nodes and Edges Definition

In [ ]:
# Nodes

def run_query_agent(state: State) -> State:
    """
    Executes a query agent with a given state.
    """
    messages = agent_chain.invoke(state)
    return {"messages": messages}


# Edges

# no custom edges

# Graph Definition

First you add the nodes of the graph.

Then the edges that link the nodes. There are two kinds of edges:
- _simple_ edge: is a directional edge that defines the connection between a start and the end nodes;
- _conditional_ edge: is a directional edge that routes the flow based on some conditions from a start node to some other nodes.

In this case we defined one conditional esge that starts from the query agent and based on the agent output decides whether to end the flow or to call a tool.

The _entry node_ defines the first node of the graph.

In [ ]:
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode, tools_condition

graph = StateGraph(State)


# Add nodes
graph.add_node("tools", ToolNode([retrieve]))
graph.add_node("query_agent", run_query_agent)


# Add edges
graph.set_entry_point("query_agent")
graph.add_conditional_edges(
    source="query_agent",  
    path=tools_condition,
)
graph.add_edge("tools", "query_agent")


# Compile
compiled_graph = graph.compile()

You can display the graph.

In [ ]:
from IPython.display import display, Image

display(
    Image(
        compiled_graph.get_graph().draw_mermaid_png(
            output_file_path="graph.png",
        )
    )
)

# Invoke

In [ ]:
result = compiled_graph.invoke(
    {"query": "Cosa significa Agile?"}
)
result

In [ ]:
print(result["messages"][-1].content)